# File Ingestion

## Setup

Using local config file (see [01_pipeline](./01_pipeline.ipynb))

In [1]:
import os
# change to the upper level folder to detect dj_local_conf.json
if os.path.basename(os.getcwd())=='notebooks': os.chdir('..')
assert os.path.basename(os.getcwd())=='adamacs', ("Please move to the main directory")
import datajoint as dj; dj.conn()
from adamacs.pipeline import subject, behavior, session, event, trial
from adamacs.ingest import behavior as ibe


[2023-07-23 16:54:19,266][INFO]: Connecting tobiasr@172.26.128.53:3306
[2023-07-23 16:54:19,313][INFO]: Connected tobiasr@172.26.128.53:3306


Manual entry

### BPod Path Setup

Your `dj.config` file should have a section for your BPod root directory under `custom`: `exp_root_data_dir`. This is a list of one or more paths where the ingestion tool will look for the relative paths it is given.

In [2]:
import datajoint as dj
from element_interface.utils import find_full_path
from adamacs.pipeline import session, event, trial
from adamacs.ingest.bpod import Bpodfile
from adamacs.paths import get_experiment_root_data_dir

bpod_path = "DB_WEZ-8701_2022-03-18_scan9FB2LN5C_sess9FB2LN5C/scan9FB2LN5C_WEZ-8701_StimArenaMaster_20220318_165447.mat"
# bpod_path = "TR_WEZ-8701_2023-01-13_scan9FG1THFD_sess9FG1TAXY/scan9FG1THFD_WEZ-8701_StimArenaMaster_20230113_172307.mat"

root_dirs = dj.config["custom"]["exp_root_data_dir"]
bpod_path_full = find_full_path(get_experiment_root_data_dir(),bpod_path)

print(f"Root: {root_dirs}\nFull: {bpod_path_full}")

Root: ['/datajoint-data/data/tobiasr']
Full: /datajoint-data/data/tobiasr/DB_WEZ-8701_2022-03-18_scan9FB2LN5C_sess9FB2LN5C/scan9FB2LN5C_WEZ-8701_StimArenaMaster_20220318_165447.mat


### Initial check of tables

In [3]:
# from adamacs.pipeline import session, event, trial

# session.Session.delete()


# print('Sessions:', len(session.Session()))
# print('Trials  :', len(trial.Trial()))
# print('Events  :', len(event.Event()))

In [4]:
# event.EventType.delete()

In [ ]:
event.BehaviorRecording()

In [ ]:
dj.Diagram(behavior)+dj.Diagram(event)+dj.Diagram(trial)

## Automated BPod ingestion

The function is designed ask for a confirmation before entered into the schema.

In [ ]:
session.Session()

In [ ]:
bpod_path = "DB_WEZ-8701_2022-03-18_scan9FB2LN5C_sess9FB2LN5C/scan9FB2LN5C_WEZ-8701_StimArenaMaster_20220318_165447.mat"
root_dirs = dj.config["custom"]["exp_root_data_dir"]
bpod_object = Bpodfile(bpod_path)




In [ ]:
bpod_object.ingest('sess9FB2LN5C', 'scan9FB2LN5C')

In [ ]:
event.Event()

In [ ]:
scansi = "scan9FJ5PTCQ"
(event.Event & f'scan_id = "{scansi}"')

In [ ]:
(trial.TrialEvent & f'scan_id = "{scansi}"' & "trial_id = 1").fetch(format = "frame", order_by = "event_start_time")

Check that insertion worked:

In [ ]:
trial.TrialEvent & 'trial_id=0'

We can also interact with bpod objects. For example:

In [ ]:
bpod_object.trial(2).events

In [ ]:
bpod_object.trial(2).attributes

In [ ]:
bpod_object.trial_data

# Add Harp recording

In [5]:
from adamacs.pipeline import behavior, event, scan
from adamacs.ingest.harp import HarpLoader, HarpLoader_sync
event_recording = event.BehaviorRecording.fetch('KEY')[0]
behavior.HarpRecording()

session_id,scan_id,harp_device_id


In [6]:
event.BehaviorRecording()

session_id,scan_id,recording_start_time,recording_duration,recording_notes
sess9FJ5PTCQ,scan9FJ5PTCQ,2023-07-20 16:01:50,320.0,


In [7]:
event_recording

{'session_id': 'sess9FJ5PTCQ', 'scan_id': 'scan9FJ5PTCQ'}

In [8]:
event.BehaviorRecording.File()

session_id,scan_id,filepath
sess9FJ5PTCQ,scan9FJ5PTCQ,/datajoint-data/data/tobiasr/RN_OPI-1681_2023-07-20_scan9FJ5PTCQ_sess9FJ5PTCQ/scan9FJ5PTCQ_RN_test_recording_0008.h5


In [9]:
scansi = "scan9FB2LN5C"
scan_key = (scan.Scan & f'scan_id = "{scansi}"').fetch('KEY')[0]

bpod_path_relative = (event.BehaviorRecording.File & scan_key).fetch1("filepath")
harp_paths = list(find_full_path(
    get_experiment_root_data_dir(), bpod_path_relative
).parent.glob("*harp*bin"))

DataJointError: fetch1 should only return one tuple. 0 tuples found

In [10]:
scansi = "scan9FJ5PTCQ"
scan_key = (scan.Scan & f'scan_id = "{scansi}"').fetch('KEY')[0]

bpod_path_relative = (event.BehaviorRecording.File & scan_key).fetch1("filepath")
harp_paths = list(find_full_path(
    get_experiment_root_data_dir(), bpod_path_relative
).parent.glob("*IMU_harp*csv"))

In [11]:
allchans = HarpLoader(harp_paths[0]).data_for_insert()

In [12]:
allchans[0]

{'channel_name': 'IMU accelerometer 1',
 'data': array([ 251,   26,  -38, ..., -169,  -14, -247]),
 'time': []}

In [13]:
scansi = "scan9FJ5PTCQ"
scan_key = (scan.Scan & f'scan_id = "{scansi}"').fetch('KEY')[0]

bpod_path_relative = (event.BehaviorRecording.File & scan_key).fetch1("filepath")
harp_paths = list(find_full_path(
    get_experiment_root_data_dir(), bpod_path_relative
).parent.glob("*2Pframes_harp*csv"))

In [14]:
allchans = HarpLoader_sync(harp_paths[0]).data_for_insert()

In [15]:
allchans[0]

{'channel_name': '2p sync',
 'data': array([0, 0, 0, ..., 0, 0, 0]),
 'time': Index([     0,      2,      4,      6,      8,     10,     12,     14,     16,
            18,
        ...
        321232, 321234, 321236, 321238, 321240, 321242, 321244, 321246, 321248,
        321250],
       dtype='int64', length=160581)}

In [16]:
behavior.HarpRecording.populate()

DuplicateError: ("Duplicate entry 'sess9FJ5PTCQ-scan9FJ5PTCQ-1' for key 'PRIMARY'", 'To ignore duplicate entries in insert, set skip_duplicates=True')

In [17]:
# event.BehaviorRecording.delete()

[2023-07-23 16:54:52,590][INFO]: Deleting 24068 rows from `tobiasr_event`.`_event`
[2023-07-23 16:54:52,598][INFO]: Deleting 1 rows from `tobiasr_event`.`behavior_recording__file`
[2023-07-23 16:54:52,605][INFO]: Deleting 1 rows from `tobiasr_event`.`behavior_recording`
[2023-07-23 16:54:58,294][WARNING]: Deletes cancelled


1

In [ ]:
bpod_path_relative

In [ ]:
bpod_path_relative 

In [ ]:
behavior.HarpRecording.Channel() & scan_key

In [ ]:
accelerometer = (behavior.HarpRecording.Channel() & "channel_name LIKE 'IMU accelerometer %'").fetch("data")
gyroscope = (behavior.HarpRecording.Channel() & "channel_name LIKE 'IMU gyroscope %'").fetch("data")
magnetometer = (behavior.HarpRecording.Channel() & "channel_name LIKE 'IMU magnetometer %'").fetch("data")

In [ ]:
import matplotlib.pyplot as plt 

fig, axes = plt.subplots(nrows=3, ncols=1, figsize=(25, 10))
# plt.rcParams['agg.path.chunksize'] = 10000  # Add this line if it does not rende


for i, arr in enumerate(accelerometer):
    axes[0].plot(arr, label=f'acellerometer {i+1}')
    
# axes[0].set_ylim([-10, 370])
axes[0].set_ylabel("accelerometer")
axes[0].set_xlabel("Time [s]")

for i, arr in enumerate(gyroscope):
    axes[1].plot(arr, label=f'gyroscope {i+1}')
    
axes[1].set_ylabel("gyroscope")
axes[1].set_xlabel("Time [s]")

for i, arr in enumerate(magnetometer):
    axes[2].plot(arr, label=f'magnetometer {i+1}')
    
# axes[2].set_ylim([-10, 370])
axes[2].set_ylabel("magnetometer")
axes[2].set_xlabel("Time [s]")

plt.show() 

In [ ]:
behavior.HarpRecording.Channel()